Authentication - you will need to upload the service account key json file when prompted


In [ ]:
from google.colab import auth
auth.authenticate_service_account()

Upload the private key for your service account.

See the guide at https://cloud.google.com/iam/docs/creating-managing-service-account-keys#iam-service-account-keys-create-console for help.




TypeError: ignored

In [ ]:

import pandas as pd
from google.cloud import bigquery

project_name='weatherlink-404323'
client = bigquery.Client()

year_start = 2015
year_stop = 2025

# Initialize a dictionary to store the schema for each table
table_schemas = {}

# Initialize a set to store common columns
common_columns_set = None

# Initialize a dictionary to store unique columns for each table
unique_columns_dict = {}

# Initialize a list to store DataFrames for each year
dfs = []

for i in range(year_start, year_stop):
    table_name = f"bigquery-public-data.nhtsa_traffic_fatalities. accident_{i}"


    try:
        # Fetch the schema (column information) for each table
        if table_name not in table_schemas:
            table = client.get_table(table_name)
            table_schemas[table_name] = set([field.name for field in table.schema])

        # If this is the first DataFrame, initialize the set with its columns
        if common_columns_set is None:
            common_columns_set = set(table_schemas[table_name])
        else:
            # Update the set to include only columns present in both DataFrames
            common_columns_set.intersection_update(table_schemas[table_name])

        # Update the set to include only columns not present in other DataFrames
        unique_columns_dict[table_name] = table_schemas[table_name].difference(common_columns_set)

        print(f"Table name {table_name} has {len(table_schemas[table_name])} cols")

        # Fetch the data and add it to a DataFrame
        query = f"SELECT * FROM `{table_name}`"
        df = client.query(query).to_dataframe()

        # Add the 'year' column to the DataFrame
        df['year'] = i

        # Calculate the FIPS code
        df['county'] = df['county'].astype(str)
        df['county'] = df['county'].str.zfill(3)
        df['state_number'] = df['state_number'].astype(str)
        df['state_number'] = df['state_number'].str.zfill(2)

        df['geoid'] = df['state_number'] + df['county']

        # Keep only the common columns
        df = df[list(common_columns_set) + ['year'] + ['geoid']]

        # Append the DataFrame to the list
        dfs.append(df)

    except Exception as e:
        print(e)
        print(f"Table {table_name} not found")

# Print unique columns for each table
for i in range(year_start, year_stop):
    table_name = f"bigquery-public-data.nhtsa_traffic_fatalities. accident_{i}"
    try:
        print(f"Unique columns in {table_name}: {unique_columns_dict[table_name]}")
    except Exception as e:
        print(e)
        print(f"Table {table_name} not found")

# Concatenate all DataFrames into a master DataFrame
master_df = pd.concat(dfs, ignore_index=True)

# Print the master DataFrame
print("\nMaster DataFrame:")
print(master_df)

Table name bigquery-public-data.nhtsa_traffic_fatalities. accident_2015 has 90 cols
Table name bigquery-public-data.nhtsa_traffic_fatalities. accident_2016 has 92 cols
Table name bigquery-public-data.nhtsa_traffic_fatalities. accident_2017 has 92 cols
Table name bigquery-public-data.nhtsa_traffic_fatalities. accident_2018 has 92 cols
Table name bigquery-public-data.nhtsa_traffic_fatalities. accident_2019 has 92 cols
Table name bigquery-public-data.nhtsa_traffic_fatalities. accident_2020 has 82 cols
404 GET https://bigquery.googleapis.com/bigquery/v2/projects/bigquery-public-data/datasets/nhtsa_traffic_fatalities/tables/%20accident_2021?prettyPrint=false: Not found: Table bigquery-public-data:nhtsa_traffic_fatalities. accident_2021
Table bigquery-public-data.nhtsa_traffic_fatalities. accident_2021 not found
404 GET https://bigquery.googleapis.com/bigquery/v2/projects/bigquery-public-data/datasets/nhtsa_traffic_fatalities/tables/%20accident_2022?prettyPrint=false: Not found: Table bigque


we are going to upload this dataframe to Google Big Query in our 'Data Warehouse', and then select a number of columns to compare for analysis.

In [ ]:
pip install pandas_gbq

In [ ]:
from pandas_gbq import to_gbq
from pandas_gbq.schema import generate_bq_schema
from io import StringIO

client = bigquery.Client()

project_id='weatherlink-404323'
dataset_id = 'weatherlink_master'
table_id = 'accident_master'

# This is a weird work around to get the dataframe acceptable for upload
# temporarily store the dataframe as a csv in a string variable
temp_csv_string = master_df.to_csv(sep=";", index=False)
temp_csv_string_IO = StringIO(temp_csv_string)
# create new dataframe from string variable
new_df = pd.read_csv(temp_csv_string_IO, sep=";")

to_gbq(new_df, f"{dataset_id}.{table_id}", project_id=project_id, if_exists='replace')


<ipython-input-20-e6e8c136e94d>:16: DtypeWarning: Columns (6,11,57,82,84) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv(temp_csv_string_IO, sep=";")
100%|██████████| 1/1 [00:00<00:00, 6765.01it/s]


Lets test by querying our new master table

In [ ]:
sql = f" SELECT COUNT(*) FROM {project_id}.{dataset_id}.{table_id}"

result = client.query(sql)

print(result)

QueryJob<project=weatherlink-404323, location=US, id=a28bd2f3-62ce-40c4-9140-54a308978395>
